# Cleaning NYC Accessible Twitter Data

## Setup

In [1]:
import pandas as pd
import re

## Read in Data

In [2]:
df = pd.read_csv('Data/nycoutages_tweets.csv')
df.shape

(3200, 2)

## Format Data

In [3]:
# reset index
df.reset_index(drop = True, inplace = True)
df.head()

,tweet_id,text
0,2017-04-01 15:18:55,FIXED: An escalator @ 59th St servicing Northb...
1,2017-04-01 15:18:49,OUTAGE: An elevator servicing Lower Mezzanine ...
2,2017-04-01 14:58:34,OUTAGE: An elevator servicing Mezzanine To Pla...
3,2017-04-01 14:28:50,OUTAGE: An escalator servicing Center Mezzanin...
4,2017-04-01 14:28:50,OUTAGE: An escalator servicing Lower Mezzanine...


In [4]:
# rename date column
df.rename(columns = {'tweet_id': 'date'}, inplace = True)
df.head()

,date,text
0,2017-04-01 15:18:55,FIXED: An escalator @ 59th St servicing Northb...
1,2017-04-01 15:18:49,OUTAGE: An elevator servicing Lower Mezzanine ...
2,2017-04-01 14:58:34,OUTAGE: An elevator servicing Mezzanine To Pla...
3,2017-04-01 14:28:50,OUTAGE: An escalator servicing Center Mezzanin...
4,2017-04-01 14:28:50,OUTAGE: An escalator servicing Lower Mezzanine...


In [5]:
# make date column as datetime format
df['date_date'] = pd.to_datetime(df['date'])
df[['date', 'date_date']].head()

,date,date_date
0,2017-04-01 15:18:55,2017-04-01 15:18:55
1,2017-04-01 15:18:49,2017-04-01 15:18:49
2,2017-04-01 14:58:34,2017-04-01 14:58:34
3,2017-04-01 14:28:50,2017-04-01 14:28:50
4,2017-04-01 14:28:50,2017-04-01 14:28:50


In [6]:
df.drop('date', axis = 1, inplace = True)
df.rename(columns = {'date_date': 'date'}, inplace = True)
df.head()

,text,date
0,FIXED: An escalator @ 59th St servicing Northb...,2017-04-01 15:18:55
1,OUTAGE: An elevator servicing Lower Mezzanine ...,2017-04-01 15:18:49
2,OUTAGE: An elevator servicing Mezzanine To Pla...,2017-04-01 14:58:34
3,OUTAGE: An escalator servicing Center Mezzanin...,2017-04-01 14:28:50
4,OUTAGE: An escalator servicing Lower Mezzanine...,2017-04-01 14:28:50


In [7]:
df.head()

,text,date
0,FIXED: An escalator @ 59th St servicing Northb...,2017-04-01 15:18:55
1,OUTAGE: An elevator servicing Lower Mezzanine ...,2017-04-01 15:18:49
2,OUTAGE: An elevator servicing Mezzanine To Pla...,2017-04-01 14:58:34
3,OUTAGE: An escalator servicing Center Mezzanin...,2017-04-01 14:28:50
4,OUTAGE: An escalator servicing Lower Mezzanine...,2017-04-01 14:28:50


## Take info out of 'text' column

### take 'status'

In [8]:
def take_status(text_str):
    text_split = text_str.split(': ')
    if len(text_split) == 1: 
        status = ''
        desc = text_str
    else: 
        status = text_split[0]
        desc = text_split[1]
    return status, desc    

In [9]:
# take out whether FIXED or OUTAGE
df['status'] = df['text'].apply(lambda x: take_status(x)[0])
df['status'].head()

0     FIXED
1    OUTAGE
2    OUTAGE
3    OUTAGE
4    OUTAGE
Name: status, dtype: object

In [10]:
# take out the description
df['desc'] = df['text'].apply(lambda x: take_status(x)[1])
df['desc'].head()

0    An escalator @ 59th St servicing Northbound Ex...
1    An elevator servicing Lower Mezzanine To Stree...
2    An elevator servicing Mezzanine To Platforms @...
3    An escalator servicing Center Mezzanine To Str...
4    An escalator servicing Lower Mezzanine To Uppe...
Name: desc, dtype: object

### Take equipment type (elevator/escalator)

In [11]:
def take_eq_type(txt): 
    txt_split = txt.split(' ')
    if (txt[:1] == 'A') | (txt[:1] == 'T'): 
        eq_type = txt_split[0] + ' ' + txt_split[1]
    else: 
        eq_type = ''
    return eq_type

In [12]:
df['human_eq_type'] = df['desc'].apply(take_eq_type)
df['human_eq_type'].head()

0    An escalator
1     An elevator
2     An elevator
3    An escalator
4    An escalator
Name: human_eq_type, dtype: object

In [13]:
df['human_eq_type'].value_counts()

An escalator     1759
An elevator      1329
That elevator     112
Name: human_eq_type, dtype: int64

### Take status from text ending

In [14]:
# take out the ending
def take_ending(txt): 
    end = txt.split(' is ')
    if len(end) > 1: 
        return end[0], end[1].strip('. ')
    else: 
        return txt, ''

In [15]:
df['ending'] = df['desc'].apply(lambda x: take_ending(x)[1])
df['ending'].head()

0        in service
1    out of service
2    out of service
3    out of service
4    out of service
Name: ending, dtype: object

In [16]:
# update description without ending
df['desc'] = df['desc'].apply(lambda x: take_ending(x)[0].strip())
df['desc'].head()

0    An escalator @ 59th St servicing Northbound Ex...
1    An elevator servicing Lower Mezzanine To Street @
2       An elevator servicing Mezzanine To Platforms @
3    An escalator servicing Center Mezzanine To Str...
4    An escalator servicing Lower Mezzanine To Uppe...
Name: desc, dtype: object

### name, servicing, location

In [17]:
def take_name(txt):
    name = txt.split('@')
    if len(name) > 1: 
        return name[0].strip(), name[1].strip()
    else: 
        return txt, ''

In [18]:
def take_svc(txt): 
    src_word = ''
    if re.search('servicing', txt):
        src_word = 'servicing'
    elif re.search('services', txt):
        src_word = 'services'
    
    if src_word != '': 
        svc = txt.split(src_word)
        return svc[0].strip(), svc[1].strip()
    else: 
        return txt, ''

In [19]:
def take_loc(txt):
    if re.search('\(.*\)', txt):
        location = re.findall('\((.*)\)', txt)[0]
        non_loc = re.sub(r'\([^)]*\)', '', txt)
    else: 
        location = ''
        non_loc = txt.strip()
    return non_loc.strip(') '), location.strip()

In [20]:
# take name from 'desc'
df['name'] = df['desc'].apply(lambda x: take_name(x)[1]).apply(lambda x: take_svc(x)[0])
df['name'].head()

0                                           59th St
1                                                  
2                                                  
3                                  Roosevelt Island
4    Jackson Hts - Roosevelt Av, 74th St - Broadway
Name: name, dtype: object

In [21]:
# take servicing from 'desc'
df['serving'] = df['desc'].apply(lambda x: take_svc(x)[1]).apply(lambda x: take_name(x)[0])
df['serving'].head()

0    Northbound Express Platform To Northbound Loca...
1                            Lower Mezzanine To Street
2                               Mezzanine To Platforms
3                           Center Mezzanine To Street
4                   Lower Mezzanine To Upper Mezzanine
Name: serving, dtype: object

In [22]:
# take location from 'name'
df['location'] = df['name'].apply(lambda x: take_loc(x)[1])
df['name'] = df['name'].apply(lambda x: take_loc(x)[0])
df[df['location'] != ''].head()

,text,date,status,desc,human_eq_type,ending,name,serving,location
7,OUTAGE: An elevator @ Forest Hills - 71st Av (...,2017-04-01 14:09:13,OUTAGE,An elevator @ Forest Hills - 71st Av (South si...,An elevator,out of service,Forest Hills - 71st Av,,South side of Queens Boulevard between 70th Ro...
19,FIXED: An elevator @ 34th St - Penn Station (S...,2017-04-01 09:38:50,FIXED,An elevator @ 34th St - Penn Station (SE corne...,An elevator,back in service,34th St - Penn Station,,SE corner of 34th Street and Eighth Avenue
22,FIXED: An elevator @ 125th St (NE corner of 12...,2017-04-01 09:28:32,FIXED,An elevator @ 125th St (NE corner of 125th Str...,An elevator,in service,125th St,Street To Mezzanine,NE corner of 125th Street and Lexington Avenue
34,FIXED: An elevator @ Jackson Hts - Roosevelt A...,2017-04-01 08:48:50,FIXED,An elevator @ Jackson Hts - Roosevelt Av&amp; ...,An elevator,back in service,Jackson Hts - Roosevelt Av&amp; 74th St - Broa...,,Roosevelt Avenue between 74th and 75th Streets
36,FIXED: An elevator @ Far Rockaway - Mott Ave (...,2017-04-01 08:08:29,FIXED,An elevator @ Far Rockaway - Mott Ave (Corner ...,An elevator,back in service,Far Rockaway - Mott Ave,,Corner of Mott Avenue and Beach 22nd Street (I...


In [23]:
# take out servicing from name
df.head(10)   

,text,date,status,desc,human_eq_type,ending,name,serving,location
0,FIXED: An escalator @ 59th St servicing Northb...,2017-04-01 15:18:55,FIXED,An escalator @ 59th St servicing Northbound Ex...,An escalator,in service,59th St,Northbound Express Platform To Northbound Loca...,
1,OUTAGE: An elevator servicing Lower Mezzanine ...,2017-04-01 15:18:49,OUTAGE,An elevator servicing Lower Mezzanine To Street @,An elevator,out of service,,Lower Mezzanine To Street,
2,OUTAGE: An elevator servicing Mezzanine To Pla...,2017-04-01 14:58:34,OUTAGE,An elevator servicing Mezzanine To Platforms @,An elevator,out of service,,Mezzanine To Platforms,
3,OUTAGE: An escalator servicing Center Mezzanin...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Center Mezzanine To Str...,An escalator,out of service,Roosevelt Island,Center Mezzanine To Street,
4,OUTAGE: An escalator servicing Lower Mezzanine...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Lower Mezzanine To Uppe...,An escalator,out of service,"Jackson Hts - Roosevelt Av, 74th St - Broadway",Lower Mezzanine To Upper Mezzanine,
5,OUTAGE: An escalator servicing Northbound Ll P...,2017-04-01 14:09:14,OUTAGE,An escalator servicing Northbound Ll Platform ...,An escalator,out of service,W 4th St - Washington Sq,Northbound Ll Platform To Northbound Ul Platform,
6,That elevator services Street To Mezzanine All...,2017-04-01 14:09:14,,That elevator services Street To Mezzanine All...,That elevator,,,Street To Mezzanine All Trains,
7,OUTAGE: An elevator @ Forest Hills - 71st Av (...,2017-04-01 14:09:13,OUTAGE,An elevator @ Forest Hills - 71st Av (South si...,An elevator,out of service,Forest Hills - 71st Av,,South side of Queens Boulevard between 70th Ro...
8,OUTAGE: An escalator servicing Mezzanine To St...,2017-04-01 14:09:13,OUTAGE,An escalator servicing Mezzanine To Street @ D...,An escalator,out of service,Delancey St - Essex St,Mezzanine To Street,
9,OUTAGE: An escalator servicing Northbound Expr...,2017-04-01 13:38:42,OUTAGE,An escalator servicing Northbound Express Plat...,An escalator,out of service,59th St,Northbound Express Platform To Northbound Loca...,


### Get the sevice information from 'that elevator'

In [24]:
# assign lag to 'serving'
df['serving_lag'] = df['serving'].shift()
df[['serving', 'serving_lag']].head()

,serving,serving_lag
0,Northbound Express Platform To Northbound Loca...,NaN
1,Lower Mezzanine To Street,Northbound Express Platform To Northbound Loca...
2,Mezzanine To Platforms,Lower Mezzanine To Street
3,Center Mezzanine To Street,Mezzanine To Platforms
4,Lower Mezzanine To Upper Mezzanine,Center Mezzanine To Street


In [25]:
df['human_eq_type_lag'] = df['human_eq_type'].shift()
df[['human_eq_type', 'human_eq_type_lag']].head()

,human_eq_type,human_eq_type_lag
0,An escalator,NaN
1,An elevator,An escalator
2,An elevator,An elevator
3,An escalator,An elevator
4,An escalator,An escalator


In [26]:
def get_svc_from_lag(row):
    if (row['human_eq_type'] == 'An elevator') & (row['human_eq_type_lag'] == 'That elevator'):
        row['serving'] = row['serving_lag']
    return row

In [27]:
df = df.apply(get_svc_from_lag, axis = 1)
df.head(10)

,text,date,status,desc,human_eq_type,ending,name,serving,location,serving_lag,human_eq_type_lag
0,FIXED: An escalator @ 59th St servicing Northb...,2017-04-01 15:18:55,FIXED,An escalator @ 59th St servicing Northbound Ex...,An escalator,in service,59th St,Northbound Express Platform To Northbound Loca...,,NaN,NaN
1,OUTAGE: An elevator servicing Lower Mezzanine ...,2017-04-01 15:18:49,OUTAGE,An elevator servicing Lower Mezzanine To Street @,An elevator,out of service,,Lower Mezzanine To Street,,Northbound Express Platform To Northbound Loca...,An escalator
2,OUTAGE: An elevator servicing Mezzanine To Pla...,2017-04-01 14:58:34,OUTAGE,An elevator servicing Mezzanine To Platforms @,An elevator,out of service,,Mezzanine To Platforms,,Lower Mezzanine To Street,An elevator
3,OUTAGE: An escalator servicing Center Mezzanin...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Center Mezzanine To Str...,An escalator,out of service,Roosevelt Island,Center Mezzanine To Street,,Mezzanine To Platforms,An elevator
4,OUTAGE: An escalator servicing Lower Mezzanine...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Lower Mezzanine To Uppe...,An escalator,out of service,"Jackson Hts - Roosevelt Av, 74th St - Broadway",Lower Mezzanine To Upper Mezzanine,,Center Mezzanine To Street,An escalator
5,OUTAGE: An escalator servicing Northbound Ll P...,2017-04-01 14:09:14,OUTAGE,An escalator servicing Northbound Ll Platform ...,An escalator,out of service,W 4th St - Washington Sq,Northbound Ll Platform To Northbound Ul Platform,,Lower Mezzanine To Upper Mezzanine,An escalator
6,That elevator services Street To Mezzanine All...,2017-04-01 14:09:14,,That elevator services Street To Mezzanine All...,That elevator,,,Street To Mezzanine All Trains,,Northbound Ll Platform To Northbound Ul Platform,An escalator
7,OUTAGE: An elevator @ Forest Hills - 71st Av (...,2017-04-01 14:09:13,OUTAGE,An elevator @ Forest Hills - 71st Av (South si...,An elevator,out of service,Forest Hills - 71st Av,Street To Mezzanine All Trains,South side of Queens Boulevard between 70th Ro...,Street To Mezzanine All Trains,That elevator
8,OUTAGE: An escalator servicing Mezzanine To St...,2017-04-01 14:09:13,OUTAGE,An escalator servicing Mezzanine To Street @ D...,An escalator,out of service,Delancey St - Essex St,Mezzanine To Street,,,An elevator
9,OUTAGE: An escalator servicing Northbound Expr...,2017-04-01 13:38:42,OUTAGE,An escalator servicing Northbound Express Plat...,An escalator,out of service,59th St,Northbound Express Platform To Northbound Loca...,,Mezzanine To Street,An escalator


In [28]:
# drop the lag columns
df.drop(['serving_lag', 'human_eq_type_lag'], axis = 1, inplace = True)

In [29]:
# drop the row with 'That elevator'
df = df[df['human_eq_type'] != 'That elevator']
df.head(10)

,text,date,status,desc,human_eq_type,ending,name,serving,location
0,FIXED: An escalator @ 59th St servicing Northb...,2017-04-01 15:18:55,FIXED,An escalator @ 59th St servicing Northbound Ex...,An escalator,in service,59th St,Northbound Express Platform To Northbound Loca...,
1,OUTAGE: An elevator servicing Lower Mezzanine ...,2017-04-01 15:18:49,OUTAGE,An elevator servicing Lower Mezzanine To Street @,An elevator,out of service,,Lower Mezzanine To Street,
2,OUTAGE: An elevator servicing Mezzanine To Pla...,2017-04-01 14:58:34,OUTAGE,An elevator servicing Mezzanine To Platforms @,An elevator,out of service,,Mezzanine To Platforms,
3,OUTAGE: An escalator servicing Center Mezzanin...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Center Mezzanine To Str...,An escalator,out of service,Roosevelt Island,Center Mezzanine To Street,
4,OUTAGE: An escalator servicing Lower Mezzanine...,2017-04-01 14:28:50,OUTAGE,An escalator servicing Lower Mezzanine To Uppe...,An escalator,out of service,"Jackson Hts - Roosevelt Av, 74th St - Broadway",Lower Mezzanine To Upper Mezzanine,
5,OUTAGE: An escalator servicing Northbound Ll P...,2017-04-01 14:09:14,OUTAGE,An escalator servicing Northbound Ll Platform ...,An escalator,out of service,W 4th St - Washington Sq,Northbound Ll Platform To Northbound Ul Platform,
7,OUTAGE: An elevator @ Forest Hills - 71st Av (...,2017-04-01 14:09:13,OUTAGE,An elevator @ Forest Hills - 71st Av (South si...,An elevator,out of service,Forest Hills - 71st Av,Street To Mezzanine All Trains,South side of Queens Boulevard between 70th Ro...
8,OUTAGE: An escalator servicing Mezzanine To St...,2017-04-01 14:09:13,OUTAGE,An escalator servicing Mezzanine To Street @ D...,An escalator,out of service,Delancey St - Essex St,Mezzanine To Street,
9,OUTAGE: An escalator servicing Northbound Expr...,2017-04-01 13:38:42,OUTAGE,An escalator servicing Northbound Express Plat...,An escalator,out of service,59th St,Northbound Express Platform To Northbound Loca...,
10,OUTAGE: An escalator servicing Center Island U...,2017-04-01 13:08:46,OUTAGE,An escalator servicing Center Island Upper Pla...,An escalator,out of service,Sutphin Blvd - Archer Av,Center Island Upper Platform To Mezzanine,


In [30]:
# confirm no more 'That elevator'
df['human_eq_type'].value_counts()

An escalator    1759
An elevator     1329
Name: human_eq_type, dtype: int64

In [31]:
# drop 'desc' & 'text' column (should be all in other columns)
df.drop(['desc', 'text'], axis = 1, inplace = True)
df.head()

,date,status,human_eq_type,ending,name,serving,location
0,2017-04-01 15:18:55,FIXED,An escalator,in service,59th St,Northbound Express Platform To Northbound Loca...,
1,2017-04-01 15:18:49,OUTAGE,An elevator,out of service,,Lower Mezzanine To Street,
2,2017-04-01 14:58:34,OUTAGE,An elevator,out of service,,Mezzanine To Platforms,
3,2017-04-01 14:28:50,OUTAGE,An escalator,out of service,Roosevelt Island,Center Mezzanine To Street,
4,2017-04-01 14:28:50,OUTAGE,An escalator,out of service,"Jackson Hts - Roosevelt Av, 74th St - Broadway",Lower Mezzanine To Upper Mezzanine,


## Clean up data

In [32]:
df.head()

,date,status,human_eq_type,ending,name,serving,location
0,2017-04-01 15:18:55,FIXED,An escalator,in service,59th St,Northbound Express Platform To Northbound Loca...,
1,2017-04-01 15:18:49,OUTAGE,An elevator,out of service,,Lower Mezzanine To Street,
2,2017-04-01 14:58:34,OUTAGE,An elevator,out of service,,Mezzanine To Platforms,
3,2017-04-01 14:28:50,OUTAGE,An escalator,out of service,Roosevelt Island,Center Mezzanine To Street,
4,2017-04-01 14:28:50,OUTAGE,An escalator,out of service,"Jackson Hts - Roosevelt Av, 74th St - Broadway",Lower Mezzanine To Upper Mezzanine,


In [33]:
# do 'status' and 'ending' always match?
pd.crosstab(df['status'], df['ending'])

ending,back in service,in service,out of service
status,,,
FIXED,102,1455,0
OUTAGE,0,0,1531


In [34]:
# these are basically same. drop ending.
df.drop('ending', axis = 1, inplace = True)

In [35]:
# How many unique station names? 
df['name'].value_counts().shape

(104,)

In [36]:
df['name'].value_counts()

                                                  373
Grand Central - 42nd St                           140
Herald Sq - 34th St                               135
125th St                                           86
Jay St - Metrotech                                 78
161st St - Yankee Stadium                          74
Bowling Green                                      72
Roosevelt Island                                   71
Jamaica Ctr - Parsons / Archer                     68
Lexington Ave - 63rd St                            63
181st St                                           61
Times Sq - 42nd St                                 54
Flushing - Main St                                 52
59th St                                            49
Gun Hill Rd                                        47
Lexington Ave - 53rd St                            47
21st St - Queensbridge                             47
Myrtle - Wyckoff Ave                               46
Atlantic Ave                

## Drop if no name (nothing much we could do about this)

In [37]:
# number of records before dropping 
df.shape[0]

3088

In [38]:
df = df[df['name'] != '']

In [39]:
# number of records after dropping
df.shape[0]

2715

## Pair up Outage and Fixed

In [40]:
df = df.sort_values(['name', 'human_eq_type', 'location', 'serving', 'date'])

In [41]:
df = df[['name', 'human_eq_type', 'location', 'serving', 'status', 'date']]
df.head(10)

,name,human_eq_type,location,serving,status,date
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12
2993,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-15 13:08:33
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37
629,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-29 09:09:04
354,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-30 18:28:44
308,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-30 21:58:46
1960,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-21 18:09:13
1910,125th St,An elevator,,Mezzanine And Uptown,FIXED,2017-03-22 00:58:46
482,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-30 04:18:39


In [42]:
# write a condition to see if they are same equipment as the record after
grp_cond = ((df.name == df.name.shift(-1)) & (df.human_eq_type == df.human_eq_type.shift(-1)) 
    & (df.location == df.location.shift(-1)) & (df.serving == df.serving.shift(-1)))

In [43]:
# mark if the the record is the same equipment as before
df['same_equip'] = False
df.loc[grp_cond, 'same_equip'] = True

In [44]:
# create leads for 'date' and 'status'
df['date_lead'] = df['date'].shift(-1)
df['status_lead'] = df['status'].shift(-1)

In [45]:
df['status2'] = ''
df['date2'] = ''

In [46]:
def get_second_status(row):
    if row['same_equip'] == True: 
        row['status2'] = row['status_lead']
        row['date2'] = row['date_lead']
    return row

In [47]:
df = df.apply(get_second_status, axis = 1)
df.head()

,name,human_eq_type,location,serving,status,date,same_equip,date_lead,status_lead,status2,date2
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,True,2017-03-15 12:18:12,OUTAGE,OUTAGE,2017-03-15 12:18:12
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,True,2017-03-15 13:08:33,FIXED,FIXED,2017-03-15 13:08:33
2993,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-15 13:08:33,True,2017-03-29 03:38:37,OUTAGE,OUTAGE,2017-03-29 03:38:37
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,True,2017-03-29 09:09:04,FIXED,FIXED,2017-03-29 09:09:04
629,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-29 09:09:04,True,2017-03-30 18:28:44,OUTAGE,OUTAGE,2017-03-30 18:28:44


In [48]:
# drop lead variables
df.drop(['date_lead', 'status_lead', 'same_equip'], axis = 1, inplace = True)
df.head()

,name,human_eq_type,location,serving,status,date,status2,date2
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,OUTAGE,2017-03-15 12:18:12
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,FIXED,2017-03-15 13:08:33
2993,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-15 13:08:33,OUTAGE,2017-03-29 03:38:37
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,FIXED,2017-03-29 09:09:04
629,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-29 09:09:04,OUTAGE,2017-03-30 18:28:44


In [49]:
df['status2_lag'] = df['status2'].shift()
df.head(10)

,name,human_eq_type,location,serving,status,date,status2,date2,status2_lag
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,OUTAGE,2017-03-15 12:18:12,NaN
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,FIXED,2017-03-15 13:08:33,OUTAGE
2993,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-15 13:08:33,OUTAGE,2017-03-29 03:38:37,FIXED
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,FIXED,2017-03-29 09:09:04,OUTAGE
629,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-29 09:09:04,OUTAGE,2017-03-30 18:28:44,FIXED
354,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-30 18:28:44,FIXED,2017-03-30 21:58:46,OUTAGE
308,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-30 21:58:46,,,FIXED
1960,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-21 18:09:13,FIXED,2017-03-22 00:58:46,
1910,125th St,An elevator,,Mezzanine And Uptown,FIXED,2017-03-22 00:58:46,OUTAGE,2017-03-30 04:18:39,FIXED
482,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-30 04:18:39,FIXED,2017-03-30 09:38:27,OUTAGE


In [50]:
df = df[(df['status'] != 'FIXED') | (df['status'] != df['status2_lag'])]
df.drop('status2_lag', axis = 1, inplace = True)
df.head(10)

,name,human_eq_type,location,serving,status,date,status2,date2
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,OUTAGE,2017-03-15 12:18:12
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,FIXED,2017-03-15 13:08:33
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,FIXED,2017-03-29 09:09:04
354,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-30 18:28:44,FIXED,2017-03-30 21:58:46
1960,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-21 18:09:13,FIXED,2017-03-22 00:58:46
482,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-30 04:18:39,FIXED,2017-03-30 09:38:27
2083,125th St,An elevator,,Mezzanine To Uptown And Downtown Platforms,OUTAGE,2017-03-21 02:18:33,FIXED,2017-03-21 06:08:45
1954,125th St,An elevator,,Mezzanine To Uptown And Downtown Platforms,OUTAGE,2017-03-21 18:48:34,FIXED,2017-03-21 21:18:43
642,125th St,An elevator,,Mezzanine To Uptown And Downtown Platforms,OUTAGE,2017-03-29 08:28:53,FIXED,2017-03-29 14:09:03
3192,125th St,An elevator,,Street To Mezzanine And Downtown Platform,FIXED,2017-03-14 16:09:00,OUTAGE,2017-03-15 05:38:21


In [51]:
# write a function to clean out status2 and date2 if they are included in the next record
def clean_status2(row): 
    if row['date_lead'] == row['date2']: 
        row['status2'] = ''
        row['date2'] = ''
    return row

In [52]:
# clean out status2 and date2 if they are included in the next record
df['date_lead'] = df['date'].shift(-1)
df = df.apply(clean_status2, axis = 1)
df.drop('date_lead', axis = 1, inplace = True)
df.head()

,name,human_eq_type,location,serving,status,date,status2,date2
3160,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,,
2999,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,FIXED,2017-03-15 13:08:33
678,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,FIXED,2017-03-29 09:09:04
354,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-30 18:28:44,FIXED,2017-03-30 21:58:46
1960,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-21 18:09:13,FIXED,2017-03-22 00:58:46


In [53]:
df.dtypes

name                     object
human_eq_type            object
location                 object
serving                  object
status                   object
date             datetime64[ns]
status2                  object
date2                    object
dtype: object

In [54]:
df['date2'] = pd.to_datetime(df['date2'])

In [55]:
df['time_spent'] = df['date2'] - df['date']
df[['date', 'date2', 'time_spent']].head()

,date,date2,time_spent
3160,2017-03-14 18:28:41,NaT,NaT
2999,2017-03-15 12:18:12,2017-03-15 13:08:33,00:50:21
678,2017-03-29 03:38:37,2017-03-29 09:09:04,05:30:27
354,2017-03-30 18:28:44,2017-03-30 21:58:46,03:30:02
1960,2017-03-21 18:09:13,2017-03-22 00:58:46,06:49:33


In [56]:
# get reset index
df.reset_index(drop = True, inplace = True)
df.head()

,name,human_eq_type,location,serving,status,date,status2,date2,time_spent
0,125th St,An elevator,,Mezzanine And Downtown,FIXED,2017-03-14 18:28:41,,NaT,NaT
1,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-15 12:18:12,FIXED,2017-03-15 13:08:33,00:50:21
2,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-29 03:38:37,FIXED,2017-03-29 09:09:04,05:30:27
3,125th St,An elevator,,Mezzanine And Downtown,OUTAGE,2017-03-30 18:28:44,FIXED,2017-03-30 21:58:46,03:30:02
4,125th St,An elevator,,Mezzanine And Uptown,OUTAGE,2017-03-21 18:09:13,FIXED,2017-03-22 00:58:46,06:49:33


In [57]:
df.to_csv('CleanedNYCAccessibleTwitterData.csv', index = False)